# Test access to minio buckets with spark

This test comes from https://github.com/marcoverl/training/blob/master/CCR/S3-from-Spark-with-oidc.ipynb

In [1]:
#!pip install liboidcagent requests xmltodict pandas boto3

In [2]:
!eval `oidc-keychain` > /dev/null && oidc-token dodas --time=3600 > /tmp/token
with open('/tmp/token') as f:
    token = f.readlines()[0].split("\n")[0]

In [3]:
import requests
import xmltodict
r = requests.post("https://minio.cloud.infn.it",
                  data={
                      'Action':
                      "AssumeRoleWithWebIdentity",
                      'Version': "2011-06-15",
                      'WebIdentityToken': token,
                      'DurationSeconds': 9000
                  },
                  verify=True)

tree = xmltodict.parse(r.content)

credentials = dict(tree['AssumeRoleWithWebIdentityResponse']
                    ['AssumeRoleWithWebIdentityResult']['Credentials'])

In [4]:
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
         .setMaster("k8s://https://kubernetes:443")
         .setAppName("MyApp")
         .set("spark.executor.memory", "1g")
         .set("spark.executor.instances", "2")
         .set("spark.kubernetes.container.image", "dodasts/spark:v3.0.1")
# configure S3 access  
         .set("spark.hadoop.fs.s3a.endpoint", "https://minio.cloud.infn.it")
         .set("spark.hadoop.fs.s3a.aws.credentials.provider","org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider")
         .set("spark.hadoop.fs.s3a.access.key", credentials["AccessKeyId"])
         .set("spark.hadoop.fs.s3a.secret.key", credentials["SecretAccessKey"]) 
         .set("spark.hadoop.fs.s3a.session.token", credentials["SessionToken"])
         .set("spark.hadoop.fs.s3a.path.style.access","true")
         .set("spark.hadoop.fs.s3a.fast.upload", "true")
         .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         .set("spark.hadoop.fs.s3a.committer.name", "directory")
       )

sc = SparkContext(conf = conf)
sc

<SparkContext master=k8s://https://kubernetes:443 appName=MyApp>

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)
# read the CSV with some options         
csv_df = spark.read.options(header='True',inferSchema='True').csv("s3a://scratch/verlato/dpc-covid19-ita-regioni.csv")
csv_df.printSchema()
csv_df.toPandas()

In [ ]:
# Test with my csv
# read the CSV with some options         
csv_df = spark.read.options(header='True',inferSchema='True').csv("s3a://legger/NLPInput/message_example.csv")
csv_df.printSchema()
csv_df.toPandas()